@dataset{
    dataset,
    author = {Timilsina, Bimal},
    year = {2021},
    month = {08},
    pages = {},
    title = {News Article Category Dataset},
}

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../input/newsarticlecategories/news-article-categories.csv')

In [3]:
display(df)

,category,title,body
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"In October 2017, Carolyn Kramer received a dis..."
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,This week I talked with actor Jeff Hiller abou...
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,The New Yorker is taking on President Donald T...
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"Kellen Hickey, a 26-year-old who lives in Huds..."
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,There’s something about combining the traditio...
...,...,...,...
6872,WOMEN,Casually Fearless: Why Millennials Are Natural...,I still think about that Tuesday night dinner ...
6873,WOMEN,Happy Birthday To Us,I remember the morning of my high school gradu...
6874,WOMEN,The Culture of Love,"My husband, Gene, doesn't wear pajamas. I aske..."
6875,WOMEN,"Carpe Diem, Oprah Style","\nBy AntonioGuillem, via ThinkStock\nBy Lisa ..."


# Prepare Data

## Remove stop words

Stop words are words that do not significantly contribute to the meaning of the text. Words like 'is', 'a', and 'the' can be removed as part of the data preparation so that the categorization can focus on the words that contribute the most meaning.

To accomplish this, we'll import the Natural Language Toolkit and then download the English stop words dataset.

In [4]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stopwords = (stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
from nltk import pos_tag, word_tokenize

def tokenize_title(title):
    tokens = word_tokenize(title)
    cleaned = []
    for t in tokens:
        if t.lower() in stopwords:
            continue
        filtered_word = re.sub(r'[\'"\s+,]', '', t).strip()
        if len(filtered_word) > 1:
            cleaned.append(filtered_word)
    return cleaned

df['tokenized'] = df['title'].map(tokenize_title)

In [6]:
categories = {}

for i, l in df.iterrows():
    category_dict = {}
    if l.category in categories:
        category_dict = categories[l['category']]
    for t in l.tokenized:
        if t not in category_dict:
            category_dict[t] = 1
        else:
            category_dict[t] += 1
    categories[l['category']] = category_dict

for category, words in categories.items():
    categories[category] = {word:count for (word, count) in words.items() if count > 2}

print(categories)

{'ARTS & CULTURE': {'Sexual': 3, 'Years': 14, 'Says': 13, 'Actor': 4, 'Jeff': 3, 'Talks': 7, 'New': 67, 'Puts': 3, 'Trump': 58, 'Racist': 3, 'Man': 8, 'Drawing': 3, 'Different': 3, 'Artist': 55, 'Gives': 3, 'Sculptures': 3, 'Modern': 7, 'Twist': 3, 'Future': 9, 'American': 23, 'Broadway': 17, 'Assault': 4, '2015': 8, 'Hair': 3, 'Ice': 4, 'Festival': 5, 'Met': 9, 'Museum': 28, 'Way': 8, 'Father': 5, 'David': 6, 'Sue': 3, 'Dead': 12, 'Cast': 3, 'Best': 11, 'Milo': 8, 'Yiannopoulos': 8, 'Art': 103, 'Shows': 11, '2017': 13, 'Let': 4, 'Books': 19, 'nt': 21, 'Read': 19, 'Holiday': 3, 'Photos': 11, 'Past': 3, 'Call': 4, 'Game': 6, 'White': 4, 'Even': 4, 'Song': 3, 'Humanity': 3, 'Money': 4, 'Photographer': 8, 'Studio': 5, 'Word': 5, 'Year': 13, 'Spike': 3, 'Got': 5, 'Show': 11, 'Black': 9, 'Free': 10, 'World': 30, 'War': 5, 'Women': 37, 'Broke': 3, 'Disney': 3, 'Club': 3, 'NYC': 8, 'Remove': 3, 'Painting': 11, 'Girl': 4, 'York': 16, 'Activist': 6, 'Wants': 5, 'Keeping': 4, 'Actual': 3, 'Asian

In [7]:
print(categories)

{'ARTS & CULTURE': {'Sexual': 3, 'Years': 14, 'Says': 13, 'Actor': 4, 'Jeff': 3, 'Talks': 7, 'New': 67, 'Puts': 3, 'Trump': 58, 'Racist': 3, 'Man': 8, 'Drawing': 3, 'Different': 3, 'Artist': 55, 'Gives': 3, 'Sculptures': 3, 'Modern': 7, 'Twist': 3, 'Future': 9, 'American': 23, 'Broadway': 17, 'Assault': 4, '2015': 8, 'Hair': 3, 'Ice': 4, 'Festival': 5, 'Met': 9, 'Museum': 28, 'Way': 8, 'Father': 5, 'David': 6, 'Sue': 3, 'Dead': 12, 'Cast': 3, 'Best': 11, 'Milo': 8, 'Yiannopoulos': 8, 'Art': 103, 'Shows': 11, '2017': 13, 'Let': 4, 'Books': 19, 'nt': 21, 'Read': 19, 'Holiday': 3, 'Photos': 11, 'Past': 3, 'Call': 4, 'Game': 6, 'White': 4, 'Even': 4, 'Song': 3, 'Humanity': 3, 'Money': 4, 'Photographer': 8, 'Studio': 5, 'Word': 5, 'Year': 13, 'Spike': 3, 'Got': 5, 'Show': 11, 'Black': 9, 'Free': 10, 'World': 30, 'War': 5, 'Women': 37, 'Broke': 3, 'Disney': 3, 'Club': 3, 'NYC': 8, 'Remove': 3, 'Painting': 11, 'Girl': 4, 'York': 16, 'Activist': 6, 'Wants': 5, 'Keeping': 4, 'Actual': 3, 'Asian